Generating constants for Kroneceker packing
===========================================

The first step is the generation of the $\Delta$ values. We generate one $\Delta$ value for each vector size $l$. The deltas are generated in ascending order for $l$, starting from $l=1$. At every step of the iteration, the $\Delta$ is assigned a number of bits $b$ that ensures that $\Delta^l$ is representable by the target integral type. When $b<3$, the iteration stops. The $\Delta$ is chosen as the first prime number $\leq \lfloor 2^b - 1 \rfloor$.


In [1]:
def create_deltas(nbits, signed):
    import gmpy2
    
    # Remove the sign bit for signed types.
    bit_width = (nbits - 1) if signed else nbits
    
    # The minimum number of bits of magnitude
    # for a Delta.
    min_nbits = 3
    
    # The first Delta (corresponding
    # to size 1) is the largest positive
    # value representable by the target
    # type (assuming two's complement for
    # signed types).
    ret = [2**bit_width - 1]
    cur_len = 2
    
    # Create the other Deltas.
    while True:
        cur_nbits = bit_width / cur_len
        
        # Check if we have enough bits.
        if int(cur_nbits) < min_nbits:
            break
        
        # Compute the candidate value.
        cand = int(2**(cur_nbits) - 1)

        # Walk back until we get a prime.
        while not gmpy2.is_prime(cand):
            cand = cand - 1

        # Double check that the product
        # of all deltas is representable
        # by the target type
        assert cand**cur_len < 2**bit_width
        
        ret.append(cand)
        
        cur_len = cur_len + 1
    
    return ret

For example, these are the deltas for signed 32-bit:

In [2]:
create_deltas(32, True)

[2147483647, 46337, 1289, 211, 71, 31, 19, 13, 7, 7]

For a given $\Delta$, the components' limits $m$ and $M$ are computed as follows:

* for signed types, $M = -m = \left( \Delta - 1 \right) / 2$ (remember that $\Delta$ is prime, thus odd),
* for unsigned types, $m = 0$ and $M = \Delta - 1$.

Note that because we are assuming two's complement, and due to the way the $\Delta$ values were created, the values of $m$ and $M$ for $l=1$ (i.e., the largest ones) are guaranteed to be representable by the target signed type. Additionally, with these choices for $\Delta$, $m$ and $M$ it can be proven that the packing and unpacking process never results in overflows.

The following function computes the value of $M$ for a given $\Delta$.

In [3]:
def delta_to_lim(delta, signed):
    # Delta is always prime, thus odd.
    assert delta % 2 != 0
    
    if signed:
        return (delta - 1) // 2
    else:
        return delta - 1

We can now move to compute and print to screen the generated $\Delta$ values, the limits on the components, and the limits on the encoded values:

In [4]:
def type_deltas(nbits, signed):
    deltas = create_deltas(nbits, signed)

    return "[{}] = {{ {} }};".format(len(deltas),", ".join(["{}{}".format(delta, "ll" if signed else "ull") for delta in deltas]))

def type_limits(nbits, signed):
    deltas = create_deltas(nbits, signed)
    
    return "[{}] = {{ {} }};".format(len(deltas),", ".join(["{}{}".format(delta_to_lim(delta, signed), "ll" if signed else "ull") for delta in deltas]))

def type_klimits(nbits, signed):
    deltas = create_deltas(nbits, signed)
    
    def delta_to_min(s, delta):
        from numpy import dot, array
        import itertools
        
        # Create the coding vector as the partial product
        # of the delta values, resulting in: [1, Delta, Delta**2, ...]
        # NOTE: need to take [:-1] because accumulate adds an element
        # at the end wrt the original vector.
        cv = array(list(itertools.accumulate([delta] * s, func = lambda a, b : a*b, initial = 1))[:-1], dtype=object)
        
        # The components limits array.
        lim = array([delta_to_lim(delta, signed)] * s, dtype=object)
        
        # The encoded value limit is the dot product
        # of cv and lim.
        return dot(cv, lim)
        
    return "[{}] = {{ {} }};".format(len(deltas),", ".join(["{}{}".format(delta_to_min(tup[0], tup[1]), "ll" if signed else "ull") for tup in enumerate(deltas, start = 1)]))

Examples for 64-bit signed integers:

In [5]:
type_deltas(64, True)

'[21] = { 9223372036854775807ll, 3037000493ll, 2097143ll, 55103ll, 6203ll, 1447ll, 509ll, 233ll, 127ll, 73ll, 47ll, 37ll, 23ll, 19ll, 17ll, 13ll, 11ll, 7ll, 7ll, 7ll, 7ll };'

In [6]:
type_limits(64, True)

'[21] = { 4611686018427387903ll, 1518500246ll, 1048571ll, 27551ll, 3101ll, 723ll, 254ll, 116ll, 63ll, 36ll, 23ll, 18ll, 11ll, 9ll, 8ll, 6ll, 5ll, 3ll, 3ll, 3ll, 3ll };'

In [7]:
type_klimits(64, True)

'[21] = { 4611686018427387903ll, 4611685997241121524ll, 4611626645054291603ll, 4609682146931245440ll, 4591757141950655621ll, 4589667151069337064ll, 4425827476873538234ll, 4343275444053330720ll, 4297377374304698943ll, 2148812914851778824ll, 1236079607542006151ll, 3291476002920017640ll, 252018180968233691ll, 399503342891442060ll, 1431211525754907896ll, 332708304591589920ll, 252723514249646885ll, 814206798955224ll, 5699447592686571ll, 39896133148806000ll, 279272932041642003ll };'

The next step is the computation of the data that allows to transform division/modulo by products of $\Delta$ values into multiplications and right shifts. This is the approach described in Figure 4.1 in:

https://gmplib.org/~tege/divcnst-pldi94.pdf

In [8]:
# Helper to compute the integer ceiling
# of the base-2 log of n.
def ceil_log2(n):
    assert n > 0
    
    bl = n.bit_length()
    
    if n & (n - 1):
        # n is not a power of 2,
        # the ceil_log2 is simply
        # the bit size.
        return bl
    else:
        # n is a power of 2,
        # the log 2 is exact and
        # corresponds to bl - 1.
        return bl - 1

# Implementation of the Figure 4.1 algorithm.
# Note that this algorithm assumes unsigned
# integrals, hence no 'signed' flag.
def divcnst_data(d, nbits):
    # Ensure that d is not zero
    # and within the limit for the
    # target type.
    assert d > 0
    assert d < 2**nbits
    
    l = ceil_log2(d)
    assert d <= 2**l
    assert 2**l <= 2*d - 1
    
    mp = 2**(nbits + l) // d - 2**nbits + 1
    assert mp < 2**nbits
    sh1 = min(l, 1)
    sh2 = max(l - 1, 0)
    assert sh1 < nbits
    assert sh2 < nbits
    assert sh2 == l - sh1
    
    return mp, sh1, sh2

# Small helper for testing that the algorithm
# is implemented correctly.
def test_div(n, d, nbits):
    mp, sh1, sh2 = divcnst_data(d, nbits)
    
    t1 = (mp * n) >> nbits
    
    ret = (n - t1) >> sh1
    ret = (t1 + ret) >> sh2
    
    return ret

Finally, here's the code for typesetting the data generated by ``divcnst_data()``. Note that the generated data is always unsigned, because the ``divcnst()`` algorithm works on unsigned types.

In [9]:
def type_divcnst_data(nbits, signed):
    deltas = create_deltas(nbits, signed)
    
    n_deltas = len(deltas)
    
    ret = []
    
    for i, delta in enumerate(deltas, start=1):
        dprod = 1
        cur = [divcnst_data(dprod, nbits)]
        
        for j in range(n_deltas):
            if j < i:
                dprod = dprod * delta
                cur.append(divcnst_data(dprod, nbits))
            else:
                cur.append((0,0,0))
        
        ret.append(cur)
    
    return "::std::tuple<??, unsigned, unsigned>[{}][{}] = {{ {} }}".format(n_deltas, n_deltas + 1, ",".join("{{ {} }}".format(",".join("{{ {}ull, {}u, {}u }}".format(tup[0], tup[1], tup[2]) for tup in row)) for row in ret))

In [10]:
type_divcnst_data(64, False)

'::std::tuple<??, unsigned, unsigned>[21][22] = { { { 1ull, 0u, 0u },{ 2ull, 1u, 63u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u } },{ { 1ull, 0u, 0u },{ 21474836506ull, 1u, 31u },{ 42949673036ull, 1u, 63u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u } },{ { 1ull, 0u, 0u },{ 10835713892937775587ull, 1u, 21u },{ 4794819346431614849ull, 1u, 42u },{ 145556515377898ull, 1u, 63u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0u },{ 0ull, 0u, 0